# Physical Attacks on Secure Systems
# Assignment 1:  Differential Power Analysis

**Goals:**
After completing these exercises successfully you should be able to perform a DPA attack and extract the key from an unprotected crypto implementation. You will have the ability to choose a suitable leakage model and a side-channel distinguisher. 

# Part A:  Tiny - AES implementation:

The target of our attack is a software implementation of [AES](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.197.pdf) that can be found on the [ChipWhisperer Github page](https://github.com/newaetech/chipwhisperer/blob/develop/hardware/victims/firmware/crypto/tiny-AES128-C/aes.c).



![Power lines](img/AESAttack.png)



## Analysis (Q1)
### DPA attack with 1-bit leakage model and DoM as distinguisher:
As discussed above, our first goal here is to attack the AES implementation using Difference of Means (DoM) as a distinguisher and value of one bit (zero, or one) as our leakage model. In order to fullfill this task you should identify the largest difference of means of all possible subkey values. First, you will get some parameter values and functions that will be used for your analysis. 

In [11]:
# Import some useful libraries
from tqdm import tnrange
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy.stats import pearsonr 
from tqdm import tqdm  
from sys import platform

/var/folders/zn/6qg4vszx34l4v669l70gq_sr0000gn/T/ipykernel_14753/363561012.py:6: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [20]:
# Load the provided data!
# ***Fill in here***
aes_traces_file_url = "" if platform == "win32" else "AES_Traces.npz"
aes_plaintext_file_url = "" if platform == "win32" else "AES_Plaintexts.npz"

with np.load(aes_traces_file_url) as data:
    traces_data = data['arr_0']
with np.load(aes_plaintext_file_url) as data:
    Plain_data = data['arr_0']

In [21]:
# How many traces are there in the AES trace set?
# ***Fill in here***
print(len(traces_data))
print((Plain_data[0].size))


2500
16


In [22]:
# How many samples per trace are there in the AES trace set?
# ***Fill in here***
print(traces_data[0].size)



4000


In [15]:
# The S-box of the AES is defined as follows
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

In [16]:
# Specify a function that gets the output of the S-Box from a plaintext and a key input.
# ***Fill in here*** 
#plaintext = 34
#k = 0 
def sbox_computation(plaintext, key):
     temp_xor = plaintext ^ key 
     return sbox[temp_xor]

Now, the goal is to separate collected traces into different groups based on a specific bit value in the result of the S-Box's output. Compute the sum of your two student numbers mod 8 to obtain the specific target bit.

In [17]:
# Our first step here will be separating our traces into different groups based on the specific bit in the S-Box's output.
# Find that specific bit by computing the sum of your student numbers mod 8. 
# ***Fill in here***
s1= 1081582
s2 = 1071237
bit = ((s1+s2) % 8)
print(bit)
def attack_target_byte(target_byte):
    DOM = []
    DOM2 = np.empty((256, 4000))
    sbox_targetbit_value =[]
    for key in tqdm(range(256)):
        traces_zero =list()
        traces_one = list()
        for i in (range(2500)):
            temp_sbox_dump = '{:08b}'.format(sbox_computation(Plain_data[i][target_byte], key))
            sbox_targetbit_value.append(temp_sbox_dump[5])
            if temp_sbox_dump[bit] == '0':
                traces_zero.append(traces_data[i])
            else:
                traces_one.append(traces_data[i])
        mean_zero = np.mean(traces_zero,axis=0)
        mean_one = np.mean(traces_one, axis=0)
        DOM2[key] = abs(mean_one - mean_zero) 
    return(DOM2)



3


In [18]:
# Then, Write loop(s) to attack subkeys one by one.
# Fill a list to hold difference of means values, and the lists for key guesses (if you have decided to save them).
# ***Fill in here*** 
key_guesses_array = []
for target_byte in tqdm(range(16)):
    DOM2 = attack_target_byte(target_byte)
    max_value = DOM2.max(axis=1)
    #print(DOM2)
    sub_key = np.where(max_value==np.amax(max_value))
    key_guesses_array.append(sub_key)
print(key_guesses_array)

100%|██████████| 16/16 [05:09<00:00, 19.33s/it]

[(array([33]),), (array([122]),), (array([37]),), (array([67]),), (array([42]),), (array([70]),), (array([45]),), (array([74]),), (array([97]),), (array([78]),), (array([100]),), (array([82]),), (array([103]),), (array([85]),), (array([107]),), (array([88]),)]


Print the recovered keys for each subkey (hex format).

In [19]:
# Print the recovered keys for each subkey (hex format).
# ***Fill in here*** 
for index in range(len(key_guesses_array)):
   print(hex(int(key_guesses_array[index][0])))

0x21
0x7a
0x25
0x43
0x2a
0x46
0x2d
0x4a
0x61
0x4e
0x64
0x52
0x67
0x55
0x6b
0x58


## Analysis (Q2)
### DPA attack with 1-bit leakage model and CPA as distinguisher:
In this section you will break AES leveraging 1-bit leakage model and CPA as distinguisher. This bit is the same bit that you have used in the previous attack. As you know from the lectures, this means that the DoM distinguisher is replaced with the Pearson correlation coefficient as a distinguisher.

In [8]:
# Build a prediction matrix that collects the output of the S-Box for all subkey guesses and the plaintexts.
# ***Fill in here***  
prediction_matrix = list()
#for target_byte in tqdm(range(16)):
for key in tqdm(range(256)):
    temp_per_key=[]
    for i in range(2500):
        #print(sbox_computation(Plain_data[i][0], key))
        temp_per_key.append(sbox_computation(Plain_data[i][0], key))
        #print(temp_per_key)
        #np.append(prediction_matrix,sbox_computation(Plain_data[i][0], key),)
    prediction_matrix.append(temp_per_key)
prediction_matrix = np.transpose(prediction_matrix)



# Then print this matrix. 
#print(prediction_matrix)
print(np.asarray(prediction_matrix).shape)
#print("Shape of prediction_matrix:", prediction_matrix.shape)  

100%|██████████| 256/256 [00:03<00:00, 76.15it/s]

(2500, 256)


In [9]:
## Compute 1-bit leakage model
# ***Fill in here*** 
leakage_model = list()
#for target_byte in tqdm(range(16)):
#for key in tqdm(range(256)):
    #bit_per_key= []
    #for i in range(2500):
        #binary_conversion= '{:08b}'.format(sbox_computation(Plain_data[i][0], key))
        #bit_per_key.append(binary_conversion[5])
    #leakage_model.append(bit_per_key)
#leakage_model = np.transpose(leakage_model)
#print(np.asarray(leakage_model).shape)


for j in range(256):
    bit_per_key = []
    for i in range(2500):
        binary_conversion= '{:08b}'.format(prediction_matrix[i][j])
        bit_per_key.append(binary_conversion[5])
    leakage_model.append(bit_per_key)
print(np.asarray(leakage_model).shape)

(256, 2500)


In [53]:
# Compute the correlation coefficients
# ***Fill in here***
# pearsoncoleration = cov(power traces, leakage mode)/ sqrt(stdd(powertraces).stdd(leakage model))
traces_data_transpose = np.transpose(traces_data)
traces_data_transpose = np.asarray(traces_data_transpose,dtype=np.float32)
leakage_model = np.asarray(leakage_model,dtype=np.int32)
print(np.asarray(traces_data_transpose).shape)
print(np.asarray(leakage_model).shape)
correlation = []
for key in tqdm(range(256)):
  temp_correlation = []
  for j in range (4000):
    pearson_coff = abs(pearsonr((np.asarray(leakage_model)[key,:]),(np.asarray(traces_data_transpose)[j,:]))[0])
    temp_correlation.append((pearson_coff))  #np.corrcoef((leakage_model[0],traces_data[j]),dtype=int)
  max_correlation_value = max(temp_correlation)
  correlation.append(max_correlation_value)

print(correlation)




(4000, 2500)
(256, 2500)


100%|██████████| 256/256 [04:28<00:00,  1.05s/it]

[0.10281479766928468, 0.08268209153398394, 0.08777690873796959, 0.09826483814760958, 0.10624800386946967, 0.09357616032812488, 0.11543593890692698, 0.09925912604340234, 0.12563319619265997, 0.09164591903340404, 0.07988258787730933, 0.12756216332237927, 0.09725991659770084, 0.11101588606214566, 0.12453074451309923, 0.1304624793629706, 0.08017061535167516, 0.10684758267787574, 0.10832002762045415, 0.11323981643150022, 0.15160459100983464, 0.09327701231009886, 0.0981725246703353, 0.07814155868408464, 0.12093717020194922, 0.08140419882524966, 0.06817246652868471, 0.10536661672226309, 0.11548751983005902, 0.0805430050367661, 0.12196650060575734, 0.09612166839153471, 0.09442264543303297, 0.35169075544773426, 0.08055671879034292, 0.07965958579141587, 0.08742251367313808, 0.13576875553754383, 0.1296556179205886, 0.08415259874435682, 0.13098085532885445, 0.07371268301757214, 0.13269778947376656, 0.10419961292950239, 0.10461973181398829, 0.1046852090920662, 0.1057777116142194, 0.1860067804491995

In [62]:
# Complete the attack
# ***Fill in here***
max_vaule_corr = np.asarray(correlation).max()
print(max_vaule_corr)
max_value_index = correlation.index(max_vaule_corr)
print(max_value_index)

0.35169075544773426
33


Print the recovered keys for each subkey (hex format).

In [ ]:
# Print the recovered key for each subkey (hex format).
# ***Fill in here***

## Analysis (Q3)


### DPA attack with Hamming Weight leakage model and CPA as distinguisher:


In [ ]:
# Define a function that computes the Hamming Weight of the sensitive value used in the attack.
# ***Fill in here***

# Use Hamming Weight leakage model
# ***Fill in here***

In [ ]:
# Compute the correlation coefficients and plot the traces [for all candidates].
# ***Fill in here***

In [ ]:
# Complete the attack
# ***Fill in here***

In [ ]:
# Print the recovered key for each subkey (hex format).
# ***Fill in here***

## Analysis (Q4)

### a) Recover all bytes of the key using Hamming Weight leakage model and CPA as distinguisher

In [1]:
# ***Fill in here*** 

### b) Finding the best place to get a good correlation between Hamming Weight and power consumption
To find the best place to get a good correlation between Hamming Weight and power consumption you can write a loop to check the correlation at each time point (or sample) and find the maximum correlation. There are many time point intervals you can use, so just pick an interval that encompasses 500-1000 points. This should give you enough points to find a good correlation. Also, since we do not care if the correlation is positive or negative we can use the absolute value of the correlation to find the maximum.

In [ ]:
# Write a a nested loop where the outer loop goes over the selected interval of samples [time points],
# and the inner loop iterates over traces.
# Calculate the Hamming Weight of the S-Box output and partition traces based on their corresponding Hamming Weights. 
# Then calculate the mean for each set of the partition and call it hw_mean_list. 
# Compute the absolute value of Pearson correlation coefficients using corr = abs(np.corrcoef(range(1,9), hw_mean_list[1:9])[0,1])
# Find the time sample that maximize the above correlation coefficient.

### c) Plot average value of measurements vs. Hamming Weight of sensitive value for the point that you found

In [ ]:
# Import some useful libraries
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(title="HW vs Power Consumption Measurement")

# Use p.line to plot HW vs Voltage Measurement.
# ***Fill in here*** 

p.xaxis.axis_label = "Hamming Weight of Sensitive Value"
p.yaxis.axis_label = "Average Value of Measurement"
show(p)

### d) Plot the same figure but for another point. What does this one look like? 

In [ ]:
# Try other points and repeat the above steps. 
# ***Fill in here*** 